#### Install the textblob library before runningg the program
pip install -U textblob

In [7]:
import pandas as pd
from textblob import TextBlob, Word
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
#from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
_wordnet = nltk.corpus.wordnet

[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [67]:
df = pd.read_csv('faq.csv')
df['Keywords_Lemma'] = ''
df

,Question,Answer,Keywords_Lemma
0,What is Bancs?,"Designed on the Digital First, Cloud First phi...",
1,What are the Key Features of TCS Bancs?,The solution supports a range of functionaliti...,
2,What is the advantage of using TCS Bancs?,The solution has helped migrate 25+ MN policie...,
3,Explanation of how Document Management works,BaNCS Business Process integrates with externa...,
4,Which distribution channels are currently supp...,NaN,
5,rocks corpora media better,NaN,
6,Please describe how your system supports proce...,BaNCS Insurance as a solution can support Spec...,
7,Please describe how your system supports proce...,TCS BaNCS allows creation and maintanenance of...,
8,Please describe how your system supports proce...,The Product workbench and business rule engine...,


In [68]:
def getLemmas(sentence):
    tags = TextBlob(sentence).tags
    #print(tags)
    lemmatized = []   
    blacklist = ['please','tcs','bancs']
    wn_lemmatizer = WordNetLemmatizer()
    
    for tag in tags:
        pos = None
        #print(tag[0], tag[1])
        word=tag[0]
        tag=tag[1]

        if tag in ("NN", "NNS", "NNP", "NNPS"):
            pos = _wordnet.NOUN
        if tag in ("JJ", "JJR", "JJS"):
            pos = _wordnet.ADJ
        if tag in ("VBN"): #("VB", "VBD", "VBG", "VBN", "VBP", "VBZ"):
            pos = _wordnet.VERB
        if tag in ("RB", "RBR", "RBS"):
            pos = _wordnet.ADV                   
        
        if pos != None and word not in blacklist:
            if pos == 'NN':
                lemmatized.append(word)
            else:
                word=wn_lemmatizer.lemmatize(word, pos)
                lemmatized.append(Word(word).lemmatize(pos))
                
    return lemmatized

#### Test getLemmas function
sentence = """Following mice attacks, caring farmers were marching to Delhi for better living conditions. 
Delhi police on Tuesday fired water cannons and teargas shells at protesting farmers as they tried to 
break barricades with their cars, automobiles and tractors."""
getLemmas(sentence)

In [69]:
df['Keywords_Lemma'] = df['Question'].apply(lambda x: getLemmas(x.lower()))
df

,Question,Answer,Keywords_Lemma
0,What is Bancs?,"Designed on the Digital First, Cloud First phi...",[]
1,What are the Key Features of TCS Bancs?,The solution supports a range of functionaliti...,"[key, feature]"
2,What is the advantage of using TCS Bancs?,The solution has helped migrate 25+ MN policie...,[advantage]
3,Explanation of how Document Management works,BaNCS Business Process integrates with externa...,"[explanation, document, management, work]"
4,Which distribution channels are currently supp...,NaN,"[distribution, channel, currently, support]"
5,rocks corpora media better,NaN,"[rock, medium, well]"
6,Please describe how your system supports proce...,BaNCS Insurance as a solution can support Spec...,"[describe, system, process, manage, special, o..."
7,Please describe how your system supports proce...,TCS BaNCS allows creation and maintanenance of...,"[describe, system, process, maintain, fund, ra..."
8,Please describe how your system supports proce...,The Product workbench and business rule engine...,"[describe, system, process, deliver, product, ..."


In [84]:
#_input='How does your system maintain fund range?'
#_input='Which parameters can be adjusted for special offers?'
_input='How does TCS Bancs support the process to Deliver Products?'
#_input = input('Enter your question:')
#_input

In [85]:
df['Score'] = 0
keywords = getLemmas(_input.lower())
for keyword in keywords:
    print(keyword)
    for idx in df.index:
        if keyword in df['Keywords_Lemma'][idx]:
            df['Score'][idx] = df['Score'][idx] + 1
            #print(df['Keywords_Lemma'][idx])
            
df.sort_values(by=['Score'],ascending=False,inplace=True)
df.reset_index(drop=True,inplace=True)
df

support
process
deliver
product


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Question,Answer,Keywords_Lemma,Score
0,Please describe how your system supports proce...,The Product workbench and business rule engine...,"[describe, system, process, deliver, product, ...",3
1,Please describe how your system supports proce...,BaNCS Insurance as a solution can support Spec...,"[describe, system, process, manage, special, o...",1
2,Please describe how your system supports proce...,TCS BaNCS allows creation and maintanenance of...,"[describe, system, process, maintain, fund, ra...",1
3,Which distribution channels are currently supp...,NaN,"[distribution, channel, currently, support]",1
4,What is Bancs?,"Designed on the Digital First, Cloud First phi...",[],0
5,What are the Key Features of TCS Bancs?,The solution supports a range of functionaliti...,"[key, feature]",0
6,What is the advantage of using TCS Bancs?,The solution has helped migrate 25+ MN policie...,[advantage],0
7,Explanation of how Document Management works,BaNCS Business Process integrates with externa...,"[explanation, document, management, work]",0
8,rocks corpora media better,NaN,"[rock, medium, well]",0


In [91]:
#df[df['Score']>0][['Question','Answer']].to_csv('output.csv')
df.head(1)[['Question','Answer']].to_csv('output.csv')

In [92]:
df.head(1)['Question']

0    Please describe how your system supports proce...
Name: Question, dtype: object

In [93]:
df.head(1)['Answer']

0    The Product workbench and business rule engine...
Name: Answer, dtype: object